In [1]:
import glob
import pandas as pd
import numpy as np
from PIL import Image

train_jpg = glob.glob('./农作物朝向检测挑战赛公开数据-初赛/train/*.jpg')
train_jpg.sort()

test_jpg = glob.glob('./农作物朝向检测挑战赛公开数据-初赛/test/*.jpg')
test_jpg.sort()

train_txt = glob.glob('./农作物朝向检测挑战赛公开数据-初赛/train/*.txt')
train_txt.sort()
train_txt = np.array([open(x).readlines() for x in train_txt]).astype(np.float32)

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

# Define the transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224 pixels
    transforms.ToTensor()  # Convert the image to a PyTorch tensor
])

model = torchvision.models.resnet18(weights='ResNet18_Weights.DEFAULT')
model.fc = torch.nn.Identity()

In [3]:
train_feat = []
for path in train_jpg:
    img = Image.open(path)
    img = transform(img)
    with torch.no_grad():
        feat = model(img[None, :, :, :])
    train_feat.append(feat)
    
train_feat = torch.vstack(train_feat)
train_feat = train_feat.data.numpy()


test_feat = []
for path in test_jpg:
    img = Image.open(path)
    img = transform(img)
    with torch.no_grad():
        feat = model(img[None, :, :, :])
    test_feat.append(feat)
    
test_feat = torch.vstack(test_feat)
test_feat = test_feat.data.numpy()

In [4]:
from sklearn.linear_model import LinearRegression
m = LinearRegression()
m.fit(train_feat, train_txt)

LinearRegression()

In [5]:
import shutil, os
if os.path.exists('submit'):
    shutil.rmtree('submit')
    
os.mkdir('submit')

for path, feat in zip(test_jpg, m.predict(test_feat)):
    up = open('./submit/' + os.path.basename(path)[:-4] + '.txt', 'w')
    for f in feat:
        up.write(str(f) + '\n')
    up.close()

In [6]:
!zip -r submit.zip submit

updating: submit/ (stored 0%)
updating: submit/00000195.txt (deflated 3%)
updating: submit/00000192.txt (stored 0%)
updating: submit/00000162.txt (stored 0%)
updating: submit/00000165.txt (stored 0%)
updating: submit/00000156.txt (stored 0%)
updating: submit/00000151.txt (stored 0%)
updating: submit/00000173.txt (stored 0%)
updating: submit/00000170.txt (stored 0%)
updating: submit/00000161.txt (stored 0%)
updating: submit/00000168.txt (deflated 3%)
updating: submit/00000159.txt (stored 0%)
updating: submit/00000176.txt (deflated 3%)
updating: submit/00000158.txt (stored 0%)
updating: submit/00000183.txt (stored 0%)
updating: submit/00000198.txt (stored 0%)
updating: submit/00000197.txt (stored 0%)
updating: submit/00000174.txt (stored 0%)
updating: submit/00000155.txt (stored 0%)
updating: submit/00000184.txt (stored 0%)
updating: submit/00000186.txt (stored 0%)
updating: submit/00000166.txt (stored 0%)
updating: submit/00000169.txt (stored 0%)
updating: submit/00000154.txt (stored 0%